In [2]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [8]:
data = pd.read_csv('stocks.csv')

In [6]:
# Dimensions of dataset
data.shape

(112457, 8)

In [11]:
# Preview dataset
print(data.head(5))
print(data.tail(5))

  Index        Date        Open  ...       Close   Adj Close  Volume
0   NYA  1965-12-31  528.690002  ...  528.690002  528.690002     0.0
1   NYA  1966-01-03  527.210022  ...  527.210022  527.210022     0.0
2   NYA  1966-01-04  527.840027  ...  527.840027  527.840027     0.0
3   NYA  1966-01-05  531.119995  ...  531.119995  531.119995     0.0
4   NYA  1966-01-06  532.070007  ...  532.070007  532.070007     0.0

[5 rows x 8 columns]
       Index        Date         Open  ...        Close    Adj Close       Volume
112452  N100  2021-05-27  1241.119995  ...  1247.069946  1247.069946  379696400.0
112453  N100  2021-05-28  1249.469971  ...  1256.599976  1256.599976  160773400.0
112454  N100  2021-05-31  1256.079956  ...  1248.930054  1248.930054   91173700.0
112455  N100  2021-06-01  1254.609985  ...  1258.579956  1258.579956  155179900.0
112456  N100  2021-06-02  1258.489990  ...  1263.619995  1263.619995  148465000.0

[5 rows x 8 columns]


In [9]:
# dataset statistics
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,110253.000000,110253.000000,110253.000000,110253.000000,110253.000000,1.102530e+05
mean,7658.515222,7704.372961,7608.000422,7657.545872,7657.351729,1.273975e+09
std,9011.478913,9066.638548,8954.506981,9011.510444,9011.608900,4.315783e+09
min,54.869999,54.869999,54.869999,54.869999,54.869999,0.000000e+00
25%,1855.030029,1864.510010,1843.979980,1855.060059,1854.179565,0.000000e+00
50%,5194.089844,5226.279785,5154.049805,5194.750000,5194.750000,4.329000e+05
75%,10134.299810,10207.820310,10060.349610,10134.830080,10134.830080,1.734314e+08
max,68775.062500,69403.750000,68516.992190,68775.062500,68775.062500,9.440374e+10
